# **Pipelines Activity**

_John Andrew Dixon_

---

**Setup**


In [208]:
# Import necessary modules

# For working with the data
import pandas as pd

# For working with NumPY arrays, the main output of sklearn
import numpy as np

# For performing a TTS
from sklearn.model_selection import train_test_split

# For scaling numerical features and encoding nominal features
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# For creating column selectors and column transformers
from sklearn.compose import make_column_selector, make_column_transformer

# For simple imputation on missing data
from sklearn.impute import SimpleImputer

# For creation of preprocessing pipelines
from sklearn.pipeline import make_pipeline

# For showing the output of sklearns display in a better way
from sklearn import set_config
set_config(display='diagram')


In [209]:
# Remote URL that has the data
url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vSdnb9XcAnl91bdZYxoJQgIapMW6SLkfr3DYGwnpBOIw-rkw-5j_3b0JLx01282OBAKVUCUJnq8OAUR/pub?output=xlsx"

# Read in the data and verify
df = pd.read_excel(url)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 16 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   name                                             77 non-null     object 
 1   Manufacturer                                     77 non-null     object 
 2   type                                             68 non-null     object 
 3   calories per serving                             70 non-null     float64
 4   grams of protein                                 77 non-null     int64  
 5   grams of fat                                     69 non-null     float64
 6   milligrams of sodium                             76 non-null     float64
 7   grams of dietary fiber                           77 non-null     float64
 8   grams of complex carbohydrates                   77 non-null     float64
 9   grams of sugars                   

---

## **Tasks**

> **Question**: _How well can the calories be predicted based on the Manufacturer, cereal type, grams of fat, grams of sugars, and weight in ounces per one serving of the cereal?_

### **Define features (X) and target (y).**
- X should only include the Manufacturer, cereal type, grams of fat, grams of sugars, and weight in ounces columns.
- y should be calories per serving

In [210]:
# Create the feature matrix (X)
columns = ["Manufacturer", "type", "grams of fat", "grams of sugars", "Weight in ounces per one serving"]
X = df.loc[:, columns]
X

,Manufacturer,type,grams of fat,grams of sugars,Weight in ounces per one serving
0,General Mills,Cold,2.0,10.0,1.00
1,General Mills,Cold,2.0,NaN,1.33
2,General Mills,Cold,2.0,1.0,1.00
3,General Mills,Cold,3.0,9.0,1.00
4,General Mills,Cold,2.0,7.0,1.00
...,...,...,...,...,...
72,Ralston Purina,Cold,NaN,2.0,1.00
73,Ralston Purina,Cold,1.0,3.0,1.00
74,American Home Food Products,Hot,1.0,NaN,1.00
75,Nabisco,Hot,0.0,0.0,1.00


In [211]:
# Create the target vector (y)
y = df["calories per serving"]
y

0     110.0
1     130.0
2       NaN
3     120.0
4     110.0
      ...  
72    110.0
73    100.0
74    100.0
75    100.0
76    100.0
Name: calories per serving, Length: 77, dtype: float64

### **Identify each feature as numerical, ordinal, or nominal.**
**Numerical**: `grams of fat`, `grams of sugars`, `Weight in ounces per one serving`

**Ordinal**: `None`

**Nominal**: `Manufacturer`, `type`

### **Train test split the data to prepare for machine learning.**

In [212]:
# Perform the Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

### **Use pipelines and column transformers to complete the following tasks:**
- Impute any missing values. Use the ‘mean’ strategy for numeric columns and the ‘most_frequent’ strategy for categorical columns.
- One-hot encode the nominal features.
    - Be sure to include the arguments: sparse=False AND handle_unknown='ignore' when creating your OneHotEncoder.
- Scale the numeric columns.

> **Note:** _In newer versions of scikit-learn, sparse is now called sparse_output._

In [213]:
# Instantiate all transformers

# Instantiate the Simple imputers for both types of columns
# Most Fequent for Nominal columns and Mean for numeric columns
most_frequent_imputer = SimpleImputer(strategy="most_frequent")
mean_imputer = SimpleImputer(strategy="mean")

# Instantiate the Scaler for scaling numerical features
scaler = StandardScaler()

# Instantiate the One-Hot Encoder
# Note: In newer versions of sklearn, sparse is now called sparse_output
one_hot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

In [214]:
# Instantiate the numeric pipeline
numeric_pipeline = make_pipeline(mean_imputer, scaler)
numeric_pipeline

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('standardscaler', StandardScaler())])

In [215]:
# Instantiate the nomical/categorical pipeline
nominal_pipeline = make_pipeline(most_frequent_imputer, one_hot_encoder)
nominal_pipeline


Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('onehotencoder',
                 OneHotEncoder(handle_unknown='ignore', sparse_output=False))])

### **All preprocessing steps should be contained within a single preprocessing object.**
- Include the arguments: remainder='drop' OR remainder='passthrough' when creating your ColumnTransformer

In [216]:
# Instantiate columns selectors
numeric_selector = make_column_selector(dtype_include="number")
nominal_selector = make_column_selector(dtype_include="object")

In [217]:
# Instantiate tuples for ColumnTransformer
numeric_tuple = (numeric_pipeline, numeric_selector)
nominal_tuple = (nominal_pipeline, nominal_selector)

In [218]:
# Instantiate a ColumnTransformer as a single preprocessing object for
# all column transformations
preprocessor = make_column_transformer(numeric_tuple, nominal_tuple, remainder="passthrough", verbose_feature_names_out=False)

### **Use your preprocessing object to transform your data appropriately, avoiding data leakage, to make it ready for modeling. Show the resulting NumPy array output.**
- The .fit() and .transform() methods should ONLY be used with the resulting preprocessing object, NOT with any individual transformer.

In [219]:
# Fit the preprocessor to ONLY the training data to prevent data leaks
preprocessor.fit(X_train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x0000025D7F10A050>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x0000025D7F10B0D0>)],
                  verbose_feature_names_out=False)

In [220]:
# Transform the X_train and X_test data
X_train_processed = preprocessor.transform(X_train)
X_test_processed = preprocessor.transform(X_test)

In [228]:
print("X_train_processed data missing values:", np.isnan(X_train_processed).sum().sum())
print("X_train_processed datatypes:", X_train_processed.dtype)
print("X_train_processed shape:", X_train_processed.shape)

print()

print("X_test_processed data missing values:", np.isnan(X_test_processed).sum().sum())
print("X_test_processed datatypes:", X_test_processed.dtype)
print("X_test_processed shape:", X_test_processed.shape)

# Output the first 5 values of the processed training data
X_train_processed

X_train_processed data missing values: 0
X_train_processed datatypes: float64
X_train_processed shape: (57, 11)

X_test_processed data missing values: 0
X_test_processed datatypes: float64
X_test_processed shape: (20, 11)


array([[-9.74679434e-01,  9.94481647e-01, -1.32764897e-01,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  1.22191915e+00,  2.03880702e+00,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00],
       [-9.74679434e-01, -8.25018407e-01, -1.32764897e-01,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  1.67679417e+00,  3.15749558e+00,
         1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00, -1.42705887e-01, -1.32764897e-01,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
  

### **Bonus: Make them into readable DataFrames**

> **Note:** _This is more for myself. It uses something I learned about in the previous core exercise after perusing some of Scikit-learn's documentation._

In [230]:
# Get the names of the features as created by the column transformer
# Ref: https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html
features = preprocessor.get_feature_names_out()

# Make each a DataFrame
X_train_processed_df = pd.DataFrame(X_train_processed, columns=features)
X_test_processed_df = pd.DataFrame(X_test_processed, columns=features)

# Display each DataFrame
display(X_train_processed_df)
X_test_processed_df

,grams of fat,grams of sugars,Weight in ounces per one serving,Manufacturer_General Mills,Manufacturer_Kelloggs,Manufacturer_Nabisco,Manufacturer_Post,Manufacturer_Quaker Oats,Manufacturer_Ralston Purina,type_Cold,type_Hot
0,-0.974679,9.944816e-01,-0.132765,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.000000,1.221919e+00,2.038807,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,-0.974679,-8.250184e-01,-0.132765,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.000000,1.676794e+00,3.157496,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.000000,-1.427059e-01,-0.132765,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
5,0.000000,-3.701434e-01,-0.132765,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
6,-0.974679,8.473162e-02,-0.132765,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
7,0.000000,7.670441e-01,-0.132765,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
8,0.000000,2.020051e-16,-0.132765,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
9,-0.974679,-8.250184e-01,-0.132765,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


,grams of fat,grams of sugars,Weight in ounces per one serving,Manufacturer_General Mills,Manufacturer_Kelloggs,Manufacturer_Nabisco,Manufacturer_Post,Manufacturer_Quaker Oats,Manufacturer_Ralston Purina,type_Cold,type_Hot
0,0.974679,8.473162e-02,-0.132765,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.974679,1.449357e+00,3.157496,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.000000,-8.250184e-01,-0.132765,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.974679,7.670441e-01,-0.132765,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.000000,-1.427059e-01,-0.132765,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
5,-0.974679,-1.507331e+00,-0.132765,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
6,0.974679,3.121691e-01,-0.132765,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
7,-0.974679,1.904232e+00,-0.132765,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
8,-0.974679,-3.701434e-01,-0.132765,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
9,-0.974679,-8.250184e-01,-0.132765,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
